In [71]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.exceptions import PreventUpdate
from dash.dependencies import Input, Output, State
import base64

In [36]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [79]:
notification_image = base64.b64encode(open("capture.png", 'rb').read())
ericsson_image = base64.b64encode(open("ericssonlogo.png", 'rb').read())

In [220]:
df = px.data.iris()  # iris is a pandas DataFrame
fig = px.scatter(df, x="sepal_width", y="sepal_length")

In [290]:

app = JupyterDash(__name__)
app.layout = html.Div([
    html.Div([
        html.Div([
            html.Div([
                html.Img(src='data:image/png;base64,{}'.format(ericsson_image.decode()), style={'height':'50px','margin-left':'-15px'}), 
                html.H1("ERICSSON DASHBOARD",style={'margin-left':'-5px','font-family': 'Montserrat, sans-serif', 'color':'#002561', 'font-weight':'900', 'font-size':'24px'})]
            ,className="HeaderLeft",style={'display':'flex','align-items':'center'}),
            html.Div([html.Img(src='data:image/png;base64,{}'.format(notification_image.decode()))]
            ,className="HeaderRight",style={'display':'flex','align-items':'center'})
            
        ],className="HeaderWrapper",style={'height':'100%','padding':'0px 20px', 'display':'flex', 'align-items':'center','justify-content':'space-between'})]
    ,className="Header",style={'width':'100%', 'height':'65px', 'background-color':'white', 'position':'fixed','border-bottom':'1px solid #002561'}),

    html.Div([
        html.Div([
            dcc.Upload(id='upload-data',
                children=html.Button(["UPLOAD FILE"], style={'font-weight':'600','letter-spacing':'0.6rem','font':'16px','background-color':'transparent','cursor':'pointer','padding':'0 20px', 'height':'32px'}),
                style={'display':'flex','justify-content':'center','flex-direction':'column', 'margin':'8px', 'margin-top':'25px'},
            # Allow multiple files to be uploaded
                multiple=False
            ),
            html.Div(id='output-data-upload'), 
        ],className="SideBarLeft", style={'display':'flex', 'flex-direction':'column','flex':'1', 'background-color': 'rgb(251, 251, 255)','height':'calc(100vh - 66px)','width':'100%'}),
       
        html.Div([
            html.Div([
                html.Div([
                    html.H1("Data Entries: 0", style={'flex':'2','margin':'25px', 'border-radius':'5px','-webkit-box-shadow': '0px 0px 15px -10px rgba(0, 0, 0, 0.75)','box-shadow':'0px 0px 15px -10px rgbe(0, 0, 0, 0.75)', 'padding': '15px', 'font-weight':'750', 'font-size':'20px', 'font-family': 'Montserrat, sans-serif', 'color':'#002561'}),
                    html.H1("Anomalies found: 0", style={'flex':'2','margin':'25px', 'border-radius':'5px','-webkit-box-shadow': '0px 0px 15px -10px rgba(0, 0, 0, 0.75)','box-shadow':'0px 0px 15px -10px rgbe(0, 0, 0, 0.75)', 'padding': '15px', 'font-weight':'750', 'font-size':'20px', 'font-family': 'Montserrat, sans-serif', 'color':'#002561'}),
                    html.H1("Anomaly Rate: 0%", style={'flex':'2','margin':'25px', 'border-radius':'5px','-webkit-box-shadow': '0px 0px 15px -10px rgba(0, 0, 0, 0.75)','box-shadow':'0px 0px 15px -10px rgbe(0, 0, 0, 0.75)', 'padding': '15px', 'font-weight':'750', 'font-size':'20px', 'font-family': 'Montserrat, sans-serif', 'color':'#002561'})
                ], className="DataWrapper", style={'display':'flex'}),
                 
                html.Div([
                    dcc.Graph(figure=fig, style={'height':'100%'})
                ], style={'flex':'6','margin-right':'25px','margin-left':'25px','margin-bottom':'25px', 'border-radius':'5px','-webkit-box-shadow': '0px 0px 15px -10px rgba(0, 0, 0, 0.75)','box-shadow':'0px 0px 15px -10px rgbe(0, 0, 0, 0.75)'})
            ],className="GraphWrapper", style={'display':'flex','flex-direction':'column','justify-content':'space-between'})
 
        ],className="HomeRight", style={'flex':'4', 'background-color':'white','height':'calc(100vh - 66px)'})
            
    ],className="HomeWrapper", style={'display':'flex', 'flex-direction':'row','width':'100%','margin-top':'66px','position':'fixed'})
    
], style={'display':'flex', 'width':'100vw', 'margin':'0 auto'})


def parse_contents(contents, filename, date):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return html.Div([
        html.H5(filename),
        html.H6(datetime.datetime.fromtimestamp(date)),

        dash_table.DataTable(
            df.to_dict('records'),
            [{'name': i, 'id': i} for i in df.columns]
        ),

        html.Hr(),  # horizontal line

        # For debugging, display the raw contents provided by the web browser
        html.Div('Raw Content'),
        html.Pre(contents[0:200] + '...', style={
            'whiteSpace': 'pre-wrap',
            'wordBreak': 'break-all'
        })
    ])

@app.callback(Output('output-data-upload', 'children'),
              Input('upload-data', 'contents'),
              State('upload-data', 'filename'),
              State('upload-data', 'last_modified'))
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)]
        return children
# Run app and display result inline in the notebook
app.run_server(mode='inline')


In [289]:
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
